# Methodology (PHL)

This notebook shows the methodology for the paper "Measuring OpenStreetMap building footprint completeness using human settlement layers", specifically for the Philippines 

Note: the notebook processes OSM completeness for June 28, 2021, and combines this with results from past years. 

## Setup

We import all of the relevant packages as well as download the datasets.

For reference, here are the original download links for the datasets:
1. High Resolution Settlement Layer (HRSL) ([Philippines](https://data.humdata.org/dataset/philippines-high-resolution-population-density-maps-demographic-estimates)) ([Madagascar](https://data.humdata.org/dataset/highresolutionpopulationdensitymaps-mdg))
2. Administrative Boundaries ([Philippines](https://data.humdata.org/dataset/philippines-administrative-levels-0-to-3)) ([Madagascar](https://data.humdata.org/dataset/madagascar-administrative-level-0-4-boundaries))
3. OpenStreetMap (OSM) ([Philippines](https://download.geofabrik.de/asia/philippines.html)) ([Madagascar](https://download.geofabrik.de/africa/madagascar.html))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import shapely
import geopandas as gpd
import rasterio
import rasterio.features

import wget

import os
import glob
from zipfile import ZipFile

In [3]:
try:
    os.mkdir("../download_data")
except Exception:
    pass

### HRSL download

Uncomment the cells below if you have not yet downloaded the HRSL datasets.

In [5]:
hrsl_phl_men_url = "https://data.humdata.org/dataset/6d9f35c0-4764-49ee-b364-329db0b7a47d/resource/5a13bb60-4506-42a5-a08a-7ccf20413179/download/phl_men_2019-06-01_geotiff.zip"
hrsl_phl_women_url = "https://data.humdata.org/dataset/6d9f35c0-4764-49ee-b364-329db0b7a47d/resource/4aff438c-43d9-47d0-853f-5a6b6ae28223/download/phl_women_2019-06-01_geotiff.zip"

In [6]:
wget.download(hrsl_phl_men_url, '../download_data/phl_hrsl_men_jun_2019.zip')

'../download_data/phl_hrsl_men_jun_2019 (1).zip'

In [7]:
wget.download(hrsl_phl_women_url, '../download_data/phl_hrsl_women_jun_2019.zip')

'../download_data/phl_hrsl_women_jun_2019 (1).zip'

### Admin boundary download

We note that the level 4 (barangay) admin boundary dataset for the Philippines is not available in the HDX website. We provide an external download link so that our results can still be reproduced.

Uncomment the cells below if you have not yet downloaded the admin boundary datasets.

In [8]:
adm_phl_url = "https://data.humdata.org/dataset/caf116df-f984-4deb-85ca-41b349d3f313/resource/12457689-6a86-4474-8032-5ca9464d38a8/download/phl_adm_psa_namria_20200529_shp.zip"

In [9]:
wget.download(adm_phl_url, '../download_data/phl_adm_all.zip')

'../download_data/phl_adm_all (1).zip'

In [10]:
adm4_phl_url = "https://storage.googleapis.com/osm-completeness-thinkingmachines/phl_adm_2015_level4_barangay.gpkg.zip"

In [11]:
wget.download(adm4_phl_url, '../download_data/phl_adm_2015_level4_barangay.gpkg.zip')

'../download_data/phl_adm_2015_level4_barangay.gpkg (1).zip'

### OSM download

Uncomment the cells below if you have not yet downloaded the OSM datasets.

In [19]:
# #### 2020 OSM Dataset
# osm_phl_url = "https://storage.googleapis.com/osm-completeness-thinkingmachines/phl_osm_jan_2020_buildings.gpkg.zip"

In [13]:
##### 2021 OSM Dataset
osm_phl_url = "https://download.geofabrik.de/asia/philippines-latest-free.shp.zip"

In [20]:
# wget.download(osm_phl_url, '../download_data/phl_osm_jan_2020_buildings.gpkg.zip')

'../download_data/phl_osm_jan_2020_buildings.gpkg.zip'

In [15]:
wget.download(osm_phl_url, '../download_data/philippines-latest-free.shp.zip')

'../download_data/philippines-latest-free.shp (1).zip'

### Unzip all datasets

In [21]:
for i in glob.glob("../download_data/*.zip"):
    if os.path.isdir(os.path.splitext(i)[0]):
        pass
    else:
        with ZipFile(i) as myzip:
            myzip.extractall(os.path.splitext(i)[0])

## What do we want our final output to look like?

In [17]:
wget.download('https://storage.googleapis.com/osm-completeness-thinkingmachines/mapthegap-phl-adm2-2021-05-29.csv', '../download_data/mapthegap-phl-adm2-2021-05-29.csv')
wget.download('https://storage.googleapis.com/osm-completeness-thinkingmachines/mapthegap-phl-adm3-2021-05-29.csv', '../download_data/mapthegap-phl-adm3-2021-05-29.csv')
wget.download('https://storage.googleapis.com/osm-completeness-thinkingmachines/mapthegap-phl-adm4-2021-05-29.csv', '../download_data/mapthegap-phl-adm4-2021-05-29.csv')

'../download_data/mapthegap-phl-adm4-2021-05-29 (1).csv'

In [18]:
province_output = pd.read_csv('../download_data/mapthegap-phl-adm2-2021-05-29.csv')

In [19]:
province_output.head()

,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021
0,Abra,PH140100000,NaN,NaN,NaN,13790,5152,72.801183,13862,5080,73.181290
1,Agusan del Norte,PH160200000,NaN,NaN,NaN,4202,35688,10.533968,15586,24304,39.072449
2,Agusan del Sur,PH160300000,NaN,NaN,NaN,2953,38147,7.184915,3422,37678,8.326034
3,Aklan,PH060400000,NaN,NaN,NaN,14693,31978,31.482077,21519,25152,46.107861
4,Albay,PH050500000,NaN,NaN,NaN,27168,31532,46.282794,32756,25944,55.802385


In [20]:
citymuni_output = pd.read_csv('../download_data/mapthegap-phl-adm3-2021-05-29.csv')

In [21]:
citymuni_output.head()

,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021
0,Aborlan,PH175301000,NaN,NaN,NaN,594,3599,14.166468,652,3541,15.549726
1,Abra de Ilog,PH175101000,NaN,NaN,NaN,1315,729,64.334638,1316,728,64.383562
2,Abucay,PH030801000,NaN,NaN,NaN,1996,646,75.548827,1993,649,75.435276
3,Abulug,PH021501000,NaN,NaN,NaN,3396,927,78.556558,3405,918,78.764747
4,Abuyog,PH083701000,NaN,NaN,NaN,1456,1005,59.162942,1473,988,59.853718


In [22]:
brgy_output = pd.read_csv('../download_data/mapthegap-phl-adm4-2021-05-29.csv')

In [23]:
brgy_output.head()

,Reg_Code,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,RURBAN,ADM4_PCODE_NAME,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,ADM4_PCODE,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021
0,110000000,REGION XI (DAVAO REGION),118200000,COMPOSTELA VALLEY,118206000.0,MAWAB,118206010.0,Sawangan,R,118206010_Sawangan,3,71,4.054054,118206010.0,6,68,8.108108
1,180000000,NEGROS ISLAND REGION (NIR),184500000,NEGROS OCCIDENTAL,184501000.0,BACOLOD CITY (Capital),184501048.0,Felisa,U,184501048_Felisa,2,272,0.729927,184501048.0,3,271,1.094891
2,120000000,REGION XII (SOCCSKSARGEN),126300000,SOUTH COTABATO,126311000.0,NORALA,126311020.0,Simsiman,R,126311020_Simsiman,116,90,56.310680,126311020.0,147,59,71.359223
3,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000.0,PANGLIMA SUGALA (BALIMBING),157001001.0,Balimbing Proper,R,157001001_Balimbing Proper,0,93,0.000000,157001001.0,5,88,5.376344
4,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000.0,PANGLIMA SUGALA (BALIMBING),157001002.0,Batu-batu (Pob.),R,157001002_Batu-batu (Pob.),11,182,5.699482,157001002.0,25,168,12.953368


## Get intersection of HRSL pixels and OSM buildings

We use Facebook’s High Resolution Settlement Layer (HRSL) as a proxy ground truth for building footprints. We then measure data completeness by getting the “percentage completeness” of pixels which is computed using the total percentage of pixels within the intersection of the human settlement layer and the OSM building footprints.

![](../assets/formula.png)

Pixels that intersect OSM buildings are *mapped*.

Pixels that do not intersect OSM buildings are *unmapped*.

### Philippines

Uncomment the cells down below if you have not yet saved the `with_buildings` and `no_buildings` GPKG files.

#### Load HRSL dataset

In [24]:
hrsl_phl_men = rasterio.open(
    "../download_data/phl_hrsl_men_jun_2019/PHL_men_2019-06-01.tif"
)

hrsl_phl_women = rasterio.open(
    "../download_data/phl_hrsl_women_jun_2019/PHL_women_2019-06-01.tif"
)

In [25]:
# hrsl_phl_crs = hrsl_phl_men.crs

#### Add male and female population to get total population

In [26]:
hrsl_phl_men_band1_mask = hrsl_phl_men.read_masks(1)

In [27]:
hrsl_phl_women_band1_mask = hrsl_phl_women.read_masks(1)

In [28]:
hrsl_phl_band1_mask = hrsl_phl_men_band1_mask + hrsl_phl_women_band1_mask

#### Convert HRSL dataset from raster to vector

In [29]:
hrsl_phl_rand = np.random.rand(
    np.shape(hrsl_phl_band1_mask)[0], np.shape(hrsl_phl_band1_mask)[1]
)
hrsl_phl_rand = hrsl_phl_rand.astype("float32")

In [30]:
hrsl_phl_band1_poly = list(
    rasterio.features.shapes(
        hrsl_phl_rand, transform=hrsl_phl_men.transform, mask=hrsl_phl_band1_mask
    )
)

In [31]:
hrsl_phl_geom = []
for geom, value in hrsl_phl_band1_poly:
    geom = shapely.geometry.shape(geom)
    hrsl_phl_geom.append(geom)

In [32]:
hrsl_phl_gdf = pd.DataFrame(hrsl_phl_geom)
hrsl_phl_gdf = gpd.GeoDataFrame(hrsl_phl_gdf, geometry=hrsl_phl_gdf[0], crs="EPSG:4326")
hrsl_phl_gdf.drop(columns=[0], inplace=True)
hrsl_phl_gdf.reset_index(level=0, inplace=True)

In [33]:
## TODO: make mkdir command for ../data

In [34]:
hrsl_phl_gdf.to_file('../data/hrsl_phl.gpkg', driver='GPKG')

#### Load OSM dataset

In [23]:
# The 2021 zip file contains many shapefiles for points of interests, landuse, waterways, etc. 
# We are only interested in loading the building footprint data
osm_phl = gpd.read_file(
    "../download_data/philippines-latest-free.shp/gis_osm_buildings_a_free_1.shp",
    driver="shp",
)

In [24]:
osm_phl

,osm_id,code,fclass,name,type,geometry
0,4350725,1500,building,None,None,"POLYGON ((120.99286 14.59423, 120.99300 14.594..."
1,4392200,1500,building,None,None,"POLYGON ((121.09786 14.63354, 121.09815 14.633..."
2,4418475,1500,building,Greenbelt 3,None,"POLYGON ((121.02102 14.55166, 121.02105 14.552..."
3,4872564,1500,building,Manila Film Center,None,"POLYGON ((120.98150 14.55072, 120.98236 14.551..."
4,4913806,1500,building,None,None,"POLYGON ((121.00756 14.45639, 121.00758 14.456..."
...,...,...,...,...,...,...
9501609,955087886,1500,building,None,None,"POLYGON ((124.59483 12.07848, 124.59488 12.078..."
9501610,955087887,1500,building,None,None,"POLYGON ((124.59450 12.07758, 124.59457 12.077..."
9501611,955087888,1500,building,None,None,"POLYGON ((124.59394 12.07685, 124.59402 12.076..."
9501612,955087889,1500,building,None,None,"POLYGON ((124.59396 12.07692, 124.59405 12.076..."


In [25]:
osm_phl.to_file('../data/philippines-latest-free.gpkg',driver = 'GPKG')

#### Get mapped pixels

In [5]:
# Just run this so you don't have to rerun everything up top
hrsl_phl_gdf = gpd.read_file('../data/hrsl_phl.gpkg', driver='GPKG')
osm_phl = gpd.read_file('../data/philippines-latest-free.gpkg',driver = 'GPKG')

In [33]:
phl_pixels_with_buildings = gpd.sjoin(
    hrsl_phl_gdf, osm_phl, how="inner", op="intersects"
)

In [34]:
# Uncomment .head() commands in final output
phl_pixels_with_buildings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7769975 entries, 1 to 5515916
Data columns (total 8 columns):
 #   Column       Dtype   
---  ------       -----   
 0   index        int64   
 1   geometry     geometry
 2   index_right  int64   
 3   osm_id       object  
 4   code         int64   
 5   fclass       object  
 6   name         object  
 7   type         object  
dtypes: geometry(1), int64(3), object(4)
memory usage: 533.5+ MB


In [35]:
phl_pixels_with_buildings = phl_pixels_with_buildings.drop_duplicates(subset='index')

In [37]:
phl_pixels_with_buildings.drop(columns=['index_right', 'osm_id', 'code', 'fclass', 'name', 'type'], inplace=True)

In [38]:
# Uncomment .head() commands in final output
phl_pixels_with_buildings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2155197 entries, 1 to 5515916
Data columns (total 2 columns):
 #   Column    Dtype   
---  ------    -----   
 0   index     int64   
 1   geometry  geometry
dtypes: geometry(1), int64(1)
memory usage: 49.3 MB


In [39]:
phl_pixels_with_buildings.to_file('../data/phl_pixels_with_buildings.gpkg', driver='GPKG')

#### Get unmapped pixels

In [4]:
# Just run this so you don't have to rerun everything up top
phl_pixels_with_buildings = gpd.read_file(
    "../data/phl_pixels_with_buildings.gpkg", driver="GPKG"
)

In [6]:
phl_pixels_no_buildings = pd.merge(hrsl_phl_gdf, phl_pixels_with_buildings, how='outer', indicator=True)

In [7]:
phl_pixels_no_buildings = phl_pixels_no_buildings[phl_pixels_no_buildings['_merge'] == 'left_only']

In [8]:
phl_pixels_no_buildings.drop(columns=['_merge'], inplace=True)

In [9]:
phl_pixels_no_buildings.to_file('../data/phl_pixels_no_buildings.gpkg', driver='GPKG')

#### Calculate percentage completeness

In 2020, it was 31.39%. What is it now?

In [10]:
len(phl_pixels_with_buildings) / (len(phl_pixels_with_buildings) + len(phl_pixels_no_buildings)) * 100

39.072296358124056

## Aggregate to different admin boundaries

### Philippines

#### Turn mapped pixels from a polygon layer to a point layer

In [5]:
# Just run this so you don't have to rerun everything up top
phl_pixels_with_buildings = gpd.read_file(
    "../data/phl_pixels_with_buildings.gpkg", driver="GPKG"
)

In [3]:
# The centroid command will give a warning but we may safely ignore it
phl_pixels_with_buildings["geometry"] = phl_pixels_with_buildings["geometry"].centroid

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


#### Turn unmapped pixels from a polygon layer to a point layer

In [4]:
# Just run this so you don't have to rerun everything up top
phl_pixels_no_buildings = gpd.read_file(
    "../data/phl_pixels_no_buildings.gpkg", driver="GPKG"
)

In [5]:
phl_pixels_no_buildings["geometry"] = phl_pixels_no_buildings["geometry"].centroid

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


#### Load level 4 admin boundary

In [3]:
phl_adm4 = gpd.read_file(
    "../download_data/phl_adm_2015_level4_barangay.gpkg/phl_adm_2015_level4_barangay.gpkg"
)

In [5]:
phl_adm4.nunique()

Reg_Code       18
Reg_Name       18
Pro_Code       87
Pro_Name       87
Mun_Code     1648
Mun_Name     1445
Bgy_Code    42037
Bgy_Name    27092
RURBAN          4
geometry    42058
dtype: int64

#### Create index for level 4 admin boundary

As shown by `nunique()` result, `Bgy_Code` and `geometry` are not 1:1, indicating that barangay codes cannot be used as a unique index. We work around this by adding a new column combining `Bgy_Code` and `Bgy_Name`

In [7]:
# Create new column indicating both the pcode and name of the barangay
phl_adm4["ADM4_PCODE_NAME"] = phl_adm4["Bgy_Code"] + "_" + phl_adm4["Bgy_Name"]

#### Find intersection of mapped pixels and level 4 admin boundary

In [8]:
# Get all the pixels that are found within the level 4 admin boundaries
phl_pixels_with_buildings_sjoin_adm4 = gpd.sjoin(
    phl_pixels_with_buildings, phl_adm4, how="left", op="within"
)

In [9]:
# Remove all columns besides index, geometry, RURBAN, and ADM4_PCODE_NAME
phl_pixels_with_buildings_sjoin_adm4.drop(
    columns=[
        "index_right",
        "Reg_Code",
        "Reg_Name",
        "Pro_Code",
        "Pro_Name",
        "Mun_Code",
        "Mun_Name",
        "Bgy_Code",
        "Bgy_Name",
    ],
    inplace=True,
)

In [10]:
# Save to file
phl_pixels_with_buildings_sjoin_adm4.to_file(
    "../data/phl_pixels_with_buildings_sjoin_adm4.gpkg", driver="GPKG"
)

#### Find intersection of unmapped pixels and level 4 admin boundary

In [11]:
phl_pixels_no_buildings_sjoin_adm4 = gpd.sjoin(
    phl_pixels_no_buildings, phl_adm4, how="left", op="within"
)

In [12]:
phl_pixels_no_buildings_sjoin_adm4.drop(
    columns=[
        "index_right",
        "Reg_Code",
        "Reg_Name",
        "Pro_Code",
        "Pro_Name",
        "Mun_Code",
        "Mun_Name",
        "Bgy_Code",
        "Bgy_Name",
    ],
    inplace=True,
)

In [27]:
phl_pixels_with_buildings_sjoin_adm4.head()

,index,geometry,RURBAN,ADM4_PCODE_NAME
0,1,POINT (121.84083 20.79833),R,020902010_Santa Rosa (Kaynatuan)
1,3,POINT (121.84389 20.79083),R,020902010_Santa Rosa (Kaynatuan)
2,4,POINT (121.84389 20.79056),R,020902010_Santa Rosa (Kaynatuan)
3,5,POINT (121.84222 20.79028),R,020902010_Santa Rosa (Kaynatuan)
4,8,POINT (121.84306 20.79000),R,020902010_Santa Rosa (Kaynatuan)


In [13]:
phl_pixels_no_buildings_sjoin_adm4.to_file(
    "../data/phl_pixels_no_buildings_sjoin_adm4.gpkg", driver="GPKG"
)

#### Load level 3 admin boundary

In [14]:
phl_adm3 = gpd.read_file(
    "../download_data/phl_adm_all/phl_admbnda_adm3_psa_namria_20200529.shp"
)

#### Find intersection of mapped pixels and level 3 admin boundary

In [15]:
phl_pixels_with_buildings_sjoin_adm3 = gpd.sjoin(
    phl_pixels_with_buildings, phl_adm3, how="left", op="within"
)

In [16]:
phl_pixels_with_buildings_sjoin_adm3.drop(
    columns=[
        "index_right",
        "Shape_Leng",
        "Shape_Area",
        "ADM3_EN",
        "ADM3_REF",
        "ADM3ALT1EN",
        "ADM3ALT2EN",
        "ADM2_EN",
        "ADM2_PCODE",
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM0_EN",
        "ADM0_PCODE",
        "date",
        "validOn",
        "validTo",
    ],
    inplace=True,
)

In [17]:
phl_pixels_with_buildings_sjoin_adm3.to_file(
    "../data/phl_pixels_with_buildings_sjoin_adm3.gpkg", driver="GPKG"
)

#### Find intersection of unmapped pixels and level 3 admin boundary

In [18]:
phl_pixels_no_buildings_sjoin_adm3 = gpd.sjoin(
    phl_pixels_no_buildings, phl_adm3, how="left", op="within"
)

In [19]:
phl_pixels_no_buildings_sjoin_adm3.drop(
    columns=[
        "index_right",
        "Shape_Leng",
        "Shape_Area",
        "ADM3_EN",
        "ADM3_REF",
        "ADM3ALT1EN",
        "ADM3ALT2EN",
        "ADM2_EN",
        "ADM2_PCODE",
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM0_EN",
        "ADM0_PCODE",
        "date",
        "validOn",
        "validTo",
    ],
    inplace=True,
)

In [20]:
phl_pixels_no_buildings_sjoin_adm3.to_file(
    "../data/phl_pixels_no_buildings_sjoin_adm3.gpkg", driver="GPKG"
)

#### Load level 2 admin boundary

In [21]:
phl_adm2 = gpd.read_file(
    "../download_data/phl_adm_all/phl_admbnda_adm2_psa_namria_20200529.shp"
)

#### Find intersection of mapped pixels and level 2 admin boundary

In [29]:
phl_pixels_with_buildings_sjoin_adm2 = gpd.sjoin(
    phl_pixels_with_buildings, phl_adm2, how="left", op="within"
)

In [31]:
phl_pixels_with_buildings_sjoin_adm2.drop(
    columns=[
        "index_right",
        "Shape_Leng",
        "Shape_Area",
        "ADM2_EN",
        "ADM2_REF",
        "ADM2ALT1EN",
        "ADM2ALT2EN",
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM0_EN",
        "ADM0_PCODE",
        "date",
        "validOn",
        "validTo",
    ],
    inplace=True,
)

In [34]:
phl_pixels_with_buildings_sjoin_adm2.to_file(
    "../data/phl_pixels_with_buildings_sjoin_adm2.gpkg", driver="GPKG"
)

#### Find intersection of unmapped pixels and level 2 admin boundary

In [35]:
phl_pixels_no_buildings_sjoin_adm2 = gpd.sjoin(
    phl_pixels_no_buildings, phl_adm2, how="left", op="within"
)

In [36]:
phl_pixels_no_buildings_sjoin_adm2.drop(
    columns=[
        "index_right",
        "Shape_Leng",
        "Shape_Area",
        "ADM2_EN",
        "ADM2_REF",
        "ADM2ALT1EN",
        "ADM2ALT2EN",
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM0_EN",
        "ADM0_PCODE",
        "date",
        "validOn",
        "validTo",
    ],
    inplace=True,
)

In [38]:
phl_pixels_no_buildings_sjoin_adm2.to_file(
    "../data/phl_pixels_no_buildings_sjoin_adm2.gpkg", driver="GPKG"
)

#### Merge and concatenate dataframes

Uncomment the cell below if you have not yet loaded these variables.

In [16]:
# # You can run this so you don't have to rerun everything up top
# phl_pixels_with_buildings_sjoin_adm4 = gpd.read_file(
#     "../data/phl_pixels_with_buildings_sjoin_adm4.gpkg", driver="GPKG"
# )
# phl_pixels_with_buildings_sjoin_adm3 = gpd.read_file(
#     "../data/phl_pixels_with_buildings_sjoin_adm3.gpkg", driver="GPKG"
# )
# phl_pixels_with_buildings_sjoin_adm2 = gpd.read_file(
#     "../data/phl_pixels_with_buildings_sjoin_adm2.gpkg", driver="GPKG"
# )
# phl_pixels_no_buildings_sjoin_adm4 = gpd.read_file(
#     "../data/phl_pixels_no_buildings_sjoin_adm4.gpkg", driver="GPKG"
# )
# phl_pixels_no_buildings_sjoin_adm3 = gpd.read_file(
#     "../data/phl_pixels_no_buildings_sjoin_adm3.gpkg", driver="GPKG"
# )
# phl_pixels_no_buildings_sjoin_adm2 = gpd.read_file(
#     "../data/phl_pixels_no_buildings_sjoin_adm2.gpkg", driver="GPKG"
# )

In [49]:
phl_pixels_with_buildings_sjoin_adm2.head()

,index,ADM2_PCODE,geometry
0,1,PH020900000,POINT (121.84083 20.79833)
1,3,PH020900000,POINT (121.84389 20.79083)
2,4,PH020900000,POINT (121.84389 20.79056)
3,5,PH020900000,POINT (121.84222 20.79028)
4,8,PH020900000,POINT (121.84306 20.79000)
...,...,...,...
2155192,5515845,PH157000000,POINT (119.47556 4.66139)
2155193,5515841,PH157000000,POINT (119.39306 4.66139)
2155194,5515843,PH157000000,POINT (119.39472 4.66139)
2155195,5515846,PH157000000,POINT (119.47556 4.66111)


In [50]:
phl_pixels_with_buildings_sjoin_adm3.head()

,index,ADM3_PCODE,geometry
0,1,PH020902000,POINT (121.84083 20.79833)
1,3,PH020902000,POINT (121.84389 20.79083)
2,4,PH020902000,POINT (121.84389 20.79056)
3,5,PH020902000,POINT (121.84222 20.79028)
4,8,PH020902000,POINT (121.84306 20.79000)


In [51]:
phl_pixels_with_buildings_sjoin_adm4.head()

,index,RURBAN,ADM4_PCODE_NAME,geometry
0,1,R,020902010_Santa Rosa (Kaynatuan),POINT (121.84083 20.79833)
1,3,R,020902010_Santa Rosa (Kaynatuan),POINT (121.84389 20.79083)
2,4,R,020902010_Santa Rosa (Kaynatuan),POINT (121.84389 20.79056)
3,5,R,020902010_Santa Rosa (Kaynatuan),POINT (121.84222 20.79028)
4,8,R,020902010_Santa Rosa (Kaynatuan),POINT (121.84306 20.79000)


### Saving final pixel output

Merging and concatenating for level 2/3/4 admin boundaries.

In [21]:
# Merging adm3 boundaries with adm4
phl_pixels_with_buildings_sjoin = phl_pixels_with_buildings_sjoin_adm3.merge(
    phl_pixels_with_buildings_sjoin_adm4[["index", "RURBAN", "ADM4_PCODE_NAME"]],
    how="left",
    left_on="index",
    right_on="index",
)

# Additionally merging adm2 to adm3 and adm4
phl_pixels_with_buildings_sjoin = phl_pixels_with_buildings_sjoin.merge(
    phl_pixels_with_buildings_sjoin_adm2[["index", "ADM2_PCODE"]],
    how="left",
    left_on="index",
    right_on="index",
)

In [26]:
# Drop nan values in the dataframe
phl_pixels_with_buildings_sjoin.dropna(
    subset=["ADM3_PCODE", "RURBAN", "ADM4_PCODE_NAME","ADM2_PCODE"], inplace=True
)

In [28]:
# Adding a column to indicate that these pixels are mapped
phl_pixels_with_buildings_sjoin["status"] = "mapped"

In [31]:
# Merging adm3 boundaries with adm4
phl_pixels_no_buildings_sjoin = phl_pixels_no_buildings_sjoin_adm3.merge(
    phl_pixels_no_buildings_sjoin_adm4[["index", "RURBAN", "ADM4_PCODE_NAME"]],
    how="left",
    left_on="index",
    right_on="index",
)

# Additionally merging adm2 to adm3 and adm4
phl_pixels_no_buildings_sjoin = phl_pixels_no_buildings_sjoin.merge(
    phl_pixels_no_buildings_sjoin_adm2[["index", "ADM2_PCODE"]],
    how="left",
    left_on="index",
    right_on="index",
)

In [32]:
# Drop nan values in the dataframe
phl_pixels_no_buildings_sjoin.dropna(
    subset=["ADM3_PCODE", "RURBAN", "ADM4_PCODE_NAME", "ADM2_PCODE"], inplace=True
)

In [33]:
# Adding a column to indicate that these pixels are unmapped
phl_pixels_no_buildings_sjoin["status"] = "unmapped"

In [44]:
# Join the mapped and unppaed pixels together
phl_pixels_all = pd.concat(
    [phl_pixels_with_buildings_sjoin, phl_pixels_no_buildings_sjoin]
)

The final output of this notebook should look like this!

In [45]:
phl_pixels_all.head()

,index,ADM3_PCODE,geometry,RURBAN,ADM4_PCODE_NAME,ADM2_PCODE,status
0,1,PH020902000,POINT (121.84083 20.79833),R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
1,3,PH020902000,POINT (121.84389 20.79083),R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
2,4,PH020902000,POINT (121.84389 20.79056),R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
3,5,PH020902000,POINT (121.84222 20.79028),R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
4,8,PH020902000,POINT (121.84306 20.79000),R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped


In [46]:
# Save all columns (including geometry) to gkpg
phl_pixels_all.to_file("../data/phl_pixels_all.gpkg", driver="GPKG")

In [47]:
# Drop geometry column
phl_pixels_all.drop(columns=["geometry"], inplace=True)

In [48]:
# Save remaining columns as csv 
phl_pixels_all.to_csv("../data/phl_pixels_all.csv", index=False)

## Calculate percent completeness per admin boundary level

In the following code snippets, we use a pivot table to get the number of mapped and unmapped pixels for each administrative region, then calculate the percent completeness

We use `ADM2_PCODE` ,`ADM3_PCODE`, and `ADM4_PCODE_Name`   as the index for each admin region.

### Loading mapped/unmapped pixels

In [3]:
# Just run this so you don't have to rerun everything up top
phl_pixels_all = pd.read_csv("../data/phl_pixels_all.csv")

### Loading admin boundaries

In [4]:
phl_adm2 = gpd.read_file(
    "../download_data/phl_adm_all/phl_admbnda_adm2_psa_namria_20200529.shp"
)

In [5]:
phl_adm3 = gpd.read_file(
    "../download_data/phl_adm_all/phl_admbnda_adm3_psa_namria_20200529.shp"
)

In [6]:
phl_adm4 = gpd.read_file(
    "../download_data/phl_adm_2015_level4_barangay.gpkg/phl_adm_2015_level4_barangay.gpkg"
)

In [50]:
phl_adm4.head()

,Reg_Code,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,RURBAN,geometry
0,110000000,REGION XI (DAVAO REGION),118200000,COMPOSTELA VALLEY,118206000,MAWAB,118206010,Sawangan,R,"MULTIPOLYGON (((125.89647 7.49963, 125.89640 7..."
1,180000000,NEGROS ISLAND REGION (NIR),184500000,NEGROS OCCIDENTAL,184501000,BACOLOD CITY (Capital),184501048,Felisa,U,"MULTIPOLYGON (((123.00528 10.60504, 123.00541 ..."
2,120000000,REGION XII (SOCCSKSARGEN),126300000,SOUTH COTABATO,126311000,NORALA,126311020,Simsiman,R,"MULTIPOLYGON (((124.66187 6.56818, 124.66324 6..."
3,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000,PANGLIMA SUGALA (BALIMBING),157001001,Balimbing Proper,R,"MULTIPOLYGON (((119.95374 5.07851, 119.95384 5..."
4,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000,PANGLIMA SUGALA (BALIMBING),157001002,Batu-batu (Pob.),R,"MULTIPOLYGON (((119.88709 5.06919, 119.88709 5..."


### Loading previous output

We will join the results from this notebook to the output from past years

In [34]:
province_output = pd.read_csv('../download_data/mapthegap-phl-adm2-2021-05-29.csv')

In [32]:
citymuni_output = pd.read_csv('../download_data/mapthegap-phl-adm3-2021-05-29.csv')

In [31]:
brgy_output = pd.read_csv('../download_data/mapthegap-phl-adm4-2021-05-29.csv')

### Pivot Table

In [11]:
# fill_value = 0 fills in NaN values with 0
# If a region has no mapped pixels, it will give NaN
# which would cause a bug in calculating pct completness
adm2_df = pd.pivot_table(phl_pixels_all[["ADM2_PCODE","status", "index"]], index = ["ADM2_PCODE"], columns = ["status"], aggfunc="count", fill_value=0)

In [12]:
adm2_df.head()

index         
status      mapped unmapped
ADM2_PCODE                 
PH012800000  45646    18479
PH012900000  54459    23735
PH013300000  32348    42918
PH015500000  17768   238488
PH020900000    998      187

The resulting dataframe has a multiindex, which we will fix in the next code blocks

In [13]:
adm2_df.columns = adm2_df.columns.get_level_values(1)
adm2_df = adm2_df.reset_index().reset_index()

In [14]:
# Dataframe now has a regular index!
adm2_df.head()

status,index,ADM2_PCODE,mapped,unmapped
0,0,PH012800000,45646,18479
1,1,PH012900000,54459,23735
2,2,PH013300000,32348,42918
3,3,PH015500000,17768,238488
4,4,PH020900000,998,187


In [15]:
# Drop the index column 
adm2_df.drop(["index"],axis = 1,inplace=True)

# Renaming columns to fit previous convention
adm2_df.rename(columns = {
    'mapped':'pixels_withbuilding_june2021',
    'unmapped':'pixels_nobuilding_june2021'
    },
    inplace=True
)

# Adding a column for percent completeness
adm2_df['percentage_completeness_june2021'] = (adm2_df['pixels_withbuilding_june2021']/(adm2_df['pixels_withbuilding_june2021'] + adm2_df['pixels_nobuilding_june2021'])) * 100

In [18]:
adm2_df.head()

status,ADM2_PCODE,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,PH012800000,45646,18479,71.182846
1,PH012900000,54459,23735,69.646009
2,PH013300000,32348,42918,42.978237
3,PH015500000,17768,238488,6.933691
4,PH020900000,998,187,84.219409


We've successfully calculated the percentage completeness for level 2. Next, we will join the data with the admin boundaries information.

In [21]:
# Create new dataframe that will store phl_adm2 
# with percentage completness output
phl_adm2_with_output = phl_adm2

# Get only the columns we need to identify region
phl_adm2_with_output = phl_adm2_with_output[[
    'ADM2_EN',
    'ADM2_PCODE',
    'ADM2_REF',
    'ADM2ALT1EN',
    'ADM2ALT2EN',
    'geometry'
 ]]

In [22]:
# Left joining percentage completness values 
# to their respective regions
phl_adm2_with_output = pd.merge(phl_adm2_with_output,adm2_df,how="left", on = "ADM2_PCODE")

In [60]:
phl_adm2_with_output.head()

,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,geometry,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,Abra,PH140100000,None,None,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",13862,5080,73.181290
1,Agusan del Norte,PH160200000,None,None,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",15586,24304,39.072449
2,Agusan del Sur,PH160300000,None,None,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",3549,37551,8.635036
3,Aklan,PH060400000,None,None,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",21777,24894,46.660667
4,Albay,PH050500000,None,None,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",32784,25916,55.850085


The last step will be to join it to the previous output and save it to file.

In [24]:
# Left joining the new percent completness values 
# to the existing values using ADM2_PCODE as index
# For the second dataframe, we only keep wanted columns
province_output_june2021 = pd.merge(
    province_output,
    phl_adm2_with_output[['ADM2_PCODE',\
                          'pixels_withbuilding_june2021',\
                          'pixels_nobuilding_june2021',\
                          'percentage_completeness_june2021',\
                          'geometry'
                         ]],
    how="left", 
    on = "ADM2_PCODE"
)

In [25]:
# Scroll through the columns to see how percentage completeness increased over time!
province_output_june2021.head()

,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021,geometry
0,Abra,PH140100000,NaN,NaN,NaN,13790,5152,72.801183,13862,5080,73.181290,13862,5080,73.181290,"POLYGON ((120.96109 17.95348, 120.97201 17.946..."
1,Agusan del Norte,PH160200000,NaN,NaN,NaN,4202,35688,10.533968,15586,24304,39.072449,15586,24304,39.072449,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9..."
2,Agusan del Sur,PH160300000,NaN,NaN,NaN,2953,38147,7.184915,3422,37678,8.326034,3549,37551,8.635036,"POLYGON ((125.88961 8.98195, 125.88896 8.96446..."
3,Aklan,PH060400000,NaN,NaN,NaN,14693,31978,31.482077,21519,25152,46.107861,21777,24894,46.660667,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ..."
4,Albay,PH050500000,NaN,NaN,NaN,27168,31532,46.282794,32756,25944,55.802385,32784,25916,55.850085,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ..."


In [26]:
# Save to .csv file
filename = "../data/mapthegap-phl-adm2-2021-06-28.csv"
province_output_june2021.to_csv(filename,index=False)

In [35]:
# Save to .gpkg file
filename = "../data/mapthegap-phl-adm2-2021-06-28.gpkg"
province_output_june2021 = gpd.GeoDataFrame(province_output_june2021,geometry='geometry')
province_output_june2021.to_file(filename,driver='GPKG')

Mostly same steps will be followed for the level 3 and 4 boundaries

### Level 3 (cities and municipalities)

In [39]:
# Pivot table of mapped and unmapped pixels
# fill_value = 0 fills in NaN values with 0
# If a region has no mapped pixels, it will give NaN
# which would cause a bug in calculating pct completness
adm3_df = pd.pivot_table(phl_pixels_all[["ADM3_PCODE","status", "index"]], index = ["ADM3_PCODE"], columns = ["status"], aggfunc="count", fill_value=0)

# Fix dataframe index
adm3_df.columns = adm3_df.columns.get_level_values(1)
adm3_df = adm3_df.reset_index().reset_index()

In [40]:
# Dataframe now has a regular index!
adm3_df.head()

status,index,ADM3_PCODE,mapped,unmapped
0,0,PH012801000,184,38
1,1,PH012802000,2965,883
2,2,PH012803000,2284,928
3,3,PH012804000,1573,334
4,4,PH012805000,3800,2193


In [41]:
# Drop the index column 
adm3_df.drop(["index"],axis = 1,inplace=True)

# Renaming columns to fit previous convention
adm3_df.rename(columns = {
    'mapped':'pixels_withbuilding_june2021',
    'unmapped':'pixels_nobuilding_june2021'
    },
    inplace=True
)

# Adding a column for percent completeness
adm3_df['percentage_completeness_june2021'] = (adm3_df['pixels_withbuilding_june2021']/(adm3_df['pixels_withbuilding_june2021'] + adm3_df['pixels_nobuilding_june2021'])) * 100

In [42]:
adm3_df.head()

status,ADM3_PCODE,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,PH012801000,184,38,82.882883
1,PH012802000,2965,883,77.053015
2,PH012803000,2284,928,71.108344
3,PH012804000,1573,334,82.485579
4,PH012805000,3800,2193,63.407309


In [43]:
# Create new dataframe that will store phl_adm3
# with percentage completness output
phl_adm3_with_output = phl_adm3

# Get only the columns we need to identify region
phl_adm3_with_output = phl_adm3_with_output[[
    'ADM3_EN',
    'ADM3_PCODE',
    'ADM3_REF',
    'ADM3ALT1EN',
    'ADM3ALT2EN',
    'geometry'
 ]]

In [44]:
# Left joining percentage completness values 
# to their respective regions
phl_adm3_with_output = pd.merge(phl_adm3_with_output,adm3_df,how="left", on = "ADM3_PCODE")

In [62]:
phl_adm3_with_output.head()

,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,geometry,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,Aborlan,PH175301000,None,None,None,"MULTIPOLYGON (((118.58350 9.37700, 118.58398 9...",652.0,3541.0,15.549726
1,Abra de Ilog,PH175101000,None,None,None,"POLYGON ((120.58412 13.50198, 120.58420 13.501...",1316.0,728.0,64.383562
2,Abucay,PH030801000,None,None,None,"POLYGON ((120.49873 14.75614, 120.49891 14.755...",1993.0,649.0,75.435276
3,Abulug,PH021501000,None,None,None,"POLYGON ((121.43455 18.46651, 121.43502 18.466...",3405.0,918.0,78.764747
4,Abuyog,PH083701000,None,None,None,"MULTIPOLYGON (((125.02684 10.73500, 125.02683 ...",1473.0,988.0,59.853718


The last step will be to join it to the previous output and save it to file.

In [46]:
# Left joining the new percent completness values 
# to the existing values using ADM3_PCODE as index
# For the second dataframe, we only keep wanted columns
citymuni_output_june2021 = pd.merge(
    citymuni_output,
    phl_adm3_with_output[['ADM3_PCODE',\
                          'pixels_withbuilding_june2021',\
                          'pixels_nobuilding_june2021',\
                          'percentage_completeness_june2021',\
                          'geometry'
                         ]],
    how="left", 
    on = "ADM3_PCODE"
)

In [47]:
# Scroll through the columns to see how percentage completeness increased over time!
citymuni_output_june2021.head()

,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021,geometry
0,Aborlan,PH175301000,NaN,NaN,NaN,594,3599,14.166468,652,3541,15.549726,652.0,3541.0,15.549726,"MULTIPOLYGON (((118.58350 9.37700, 118.58398 9..."
1,Abra de Ilog,PH175101000,NaN,NaN,NaN,1315,729,64.334638,1316,728,64.383562,1316.0,728.0,64.383562,"POLYGON ((120.58412 13.50198, 120.58420 13.501..."
2,Abucay,PH030801000,NaN,NaN,NaN,1996,646,75.548827,1993,649,75.435276,1993.0,649.0,75.435276,"POLYGON ((120.49873 14.75614, 120.49891 14.755..."
3,Abulug,PH021501000,NaN,NaN,NaN,3396,927,78.556558,3405,918,78.764747,3405.0,918.0,78.764747,"POLYGON ((121.43455 18.46651, 121.43502 18.466..."
4,Abuyog,PH083701000,NaN,NaN,NaN,1456,1005,59.162942,1473,988,59.853718,1473.0,988.0,59.853718,"MULTIPOLYGON (((125.02684 10.73500, 125.02683 ..."


In [48]:
# Save to .csv file
filename = "../data/mapthegap-phl-adm3-2021-06-28.csv"
citymuni_output_june2021.to_csv(filename)

In [49]:
# Save to .gpkg file
filename = "../data/mapthegap-phl-adm3-2021-06-28.gpkg"
citymuni_output_june2021 = gpd.GeoDataFrame(citymuni_output_june2021,geometry='geometry')
citymuni_output_june2021.to_file(filename,driver='GPKG')

### Level 4 (barangays)

Note: Instead of using PCODE, we use `ADM4_PCODE_NAME` for this section

In [51]:
# Pivot table of mapped and unmapped pixels
# fill_value = 0 fills in NaN values with 0
# If a region has no mapped pixels, it will give NaN
# which would cause a bug in calculating pct completness
adm4_df = pd.pivot_table(phl_pixels_all[["ADM4_PCODE_NAME","status", "index"]], index = ["ADM4_PCODE_NAME"], columns = ["status"], aggfunc="count", fill_value=0)

# Fix dataframe index
adm4_df.columns = adm4_df.columns.get_level_values(1)
adm4_df = adm4_df.reset_index().reset_index()

In [52]:
# Dataframe now has a regular index!
adm4_df.head()

status,index,ADM4_PCODE_NAME,mapped,unmapped
0,0,012801001_Adams (Pob.),184,38
1,1,012802001_Bani,89,52
2,2,012802002_Buyon,143,54
3,3,012802003_Cabaruan,151,56
4,4,012802004_Cabulalaan,80,21


In [53]:
# Drop the index column 
adm4_df.drop(["index"],axis = 1,inplace=True)

# Renaming columns to fit previous convention
adm4_df.rename(columns = {
    'mapped':'pixels_withbuilding_june2021',
    'unmapped':'pixels_nobuilding_june2021'
    },
    inplace=True
)

# Adding a column for percent completeness
adm4_df['percentage_completeness_june2021'] = (adm4_df['pixels_withbuilding_june2021']/(adm4_df['pixels_withbuilding_june2021'] + adm4_df['pixels_nobuilding_june2021'])) * 100

In [54]:
adm4_df.head()

status,ADM4_PCODE_NAME,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,012801001_Adams (Pob.),184,38,82.882883
1,012802001_Bani,89,52,63.120567
2,012802002_Buyon,143,54,72.588832
3,012802003_Cabaruan,151,56,72.946860
4,012802004_Cabulalaan,80,21,79.207921


In [55]:
# Create new dataframe that will store phl_adm4
# with percentage completness output
phl_adm4_with_output = phl_adm4

# Get only the columns we need to identify region
phl_adm4_with_output = phl_adm4_with_output[[
    'Reg_Code',
    'Reg_Name',
    'Pro_Code',
    'Bgy_Code',
    'Bgy_Name',
    'RURBAN',
    'geometry'
 ]]

# Create new column indicating both the pcode and name of the barangay
phl_adm4_with_output["ADM4_PCODE_NAME"] = phl_adm4_with_output["Bgy_Code"] + "_" + phl_adm4_with_output["Bgy_Name"]

/opt/conda/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [56]:
# Left joining percentage completness values 
# to their respective regions
phl_adm4_with_output = pd.merge(phl_adm4_with_output,adm4_df,how="left", on = "ADM4_PCODE_NAME")

In [58]:
phl_adm4_with_output.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 42058 entries, 0 to 42057
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Reg_Code                          42058 non-null  object  
 1   Reg_Name                          42058 non-null  object  
 2   Pro_Code                          42058 non-null  object  
 3   Bgy_Code                          42058 non-null  object  
 4   Bgy_Name                          42058 non-null  object  
 5   RURBAN                            42058 non-null  object  
 6   geometry                          42058 non-null  geometry
 7   ADM4_PCODE_NAME                   42058 non-null  object  
 8   pixels_withbuilding_june2021      41697 non-null  float64 
 9   pixels_nobuilding_june2021        41697 non-null  float64 
 10  percentage_completeness_june2021  41697 non-null  float64 
dtypes: float64(3), geometry(1), object(7)
memory u

The last step will be to join it to the previous output and save it to file.

In [63]:
# Left joining the new percent completness values 
# to the existing values using ADM3_PCODE as index
# For the second dataframe, we only keep wanted columns
brgy_output_june2021 = pd.merge(
    brgy_output,
    phl_adm4_with_output[['ADM4_PCODE_NAME',\
                          'pixels_withbuilding_june2021',\
                          'pixels_nobuilding_june2021',\
                          'percentage_completeness_june2021',\
                          'geometry'
                         ]],
    how="left", 
    on = "ADM4_PCODE_NAME"
)

In [66]:
# Scroll through the columns to see how percentage completeness increased over time!
brgy_output_june2021.head()

,Reg_Code,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,RURBAN,ADM4_PCODE_NAME,...,pixels_nobuilding_june2020,percentage_completeness_june2020,ADM4_PCODE,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021,geometry
0,110000000,REGION XI (DAVAO REGION),118200000,COMPOSTELA VALLEY,118206000.0,MAWAB,118206010.0,Sawangan,R,118206010_Sawangan,...,71,4.054054,118206010.0,6,68,8.108108,6.0,68.0,8.108108,"MULTIPOLYGON (((125.89647 7.49963, 125.89640 7..."
1,180000000,NEGROS ISLAND REGION (NIR),184500000,NEGROS OCCIDENTAL,184501000.0,BACOLOD CITY (Capital),184501048.0,Felisa,U,184501048_Felisa,...,272,0.729927,184501048.0,3,271,1.094891,3.0,271.0,1.094891,"MULTIPOLYGON (((123.00528 10.60504, 123.00541 ..."
2,120000000,REGION XII (SOCCSKSARGEN),126300000,SOUTH COTABATO,126311000.0,NORALA,126311020.0,Simsiman,R,126311020_Simsiman,...,90,56.310680,126311020.0,147,59,71.359223,147.0,59.0,71.359223,"MULTIPOLYGON (((124.66187 6.56818, 124.66324 6..."
3,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000.0,PANGLIMA SUGALA (BALIMBING),157001001.0,Balimbing Proper,R,157001001_Balimbing Proper,...,93,0.000000,157001001.0,5,88,5.376344,5.0,88.0,5.376344,"MULTIPOLYGON (((119.95374 5.07851, 119.95384 5..."
4,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000.0,PANGLIMA SUGALA (BALIMBING),157001002.0,Batu-batu (Pob.),R,157001002_Batu-batu (Pob.),...,182,5.699482,157001002.0,25,168,12.953368,25.0,168.0,12.953368,"MULTIPOLYGON (((119.88709 5.06919, 119.88709 5..."


In [67]:
# Save to .csv file
filename = "../data/mapthegap-phl-adm4-2021-06-28.csv"
brgy_output_june2021.to_csv(filename)

In [68]:
# Save to .gpkg file
filename = "../data/mapthegap-phl-adm4-2021-06-28.gpkg"
brgy_output_june2021 = gpd.GeoDataFrame(brgy_output_june2021,geometry='geometry')
brgy_output_june2021.to_file(filename,driver='GPKG')

# Finish!
By this point, you should already have the percent completeness for the whole country and per admin region.

1. `phl_pixels_all.csv` and `phl_pixels_all.gpkg` which contains mapped/unmapped pixels, labelled by admin boundary PCODES
2. `mapthegap-phl-adm<2/3/4>-2021-06-28.csv` and `mapthegap-phl-adm<2/3/4>-2021-06-28.gpkg`, which contains percent completeness values per administrative region.

To convert this into tilesets for plotting, refer to `2_CreateTileset.ipynb`